In [1]:
import os
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"
from mpi4py import MPI
import pickle


# torch
from torch.nn.parameter import Parameter
import torch
import torch.nn as nn
torch.autograd.set_detect_anomaly(False)

# quimb
import quimb as qu
import quimb.tensor as qtn
import autoray as ar
from autoray import do

# from vmc_torch.experiment.tn_model import *
from vmc_torch.sampler import MetropolisExchangeSamplerSpinful
from vmc_torch.variational_state import Variational_State
from vmc_torch.optimizer import SGD, SignedSGD, SignedRandomSGD, SR, TrivialPreconditioner, Adam, SGD_momentum, DecayScheduler
from vmc_torch.VMC import VMC
# from vmc_torch.hamiltonian import spinful_Fermi_Hubbard_square_lattice
from vmc_torch.hamiltonian_torch import spinful_Fermi_Hubbard_square_lattice_torch
from vmc_torch.torch_utils import SVD,QR
from vmc_torch.fermion_utils import generate_random_fpeps
from vmc_torch.utils import closest_divisible

# Register safe SVD and QR functions to torch
ar.register_function('torch','linalg.svd',SVD.apply)
ar.register_function('torch','linalg.qr',QR.apply)

from vmc_torch.global_var import DEBUG


COMM = MPI.COMM_WORLD
SIZE = COMM.Get_size()
RANK = COMM.Get_rank()

# Hamiltonian parameters
Lx = int(4)
Ly = int(4)
symmetry = 'Z2'
t = 1.0
U = 8.0
N_f = int(Lx*Ly-2)
# N_f=12
n_fermions_per_spin = (N_f//2, N_f//2)
H = spinful_Fermi_Hubbard_square_lattice_torch(Lx, Ly, t, U, N_f, pbc=False, n_fermions_per_spin=n_fermions_per_spin)
graph = H.graph
# TN parameters
D = 4
chi = 16
dtype=torch.float64

# Load PEPS
try:
    skeleton = pickle.load(open(f"../../data/{Lx}x{Ly}/t={t}_U={U}/N={N_f}/{symmetry}/D={D}/peps_skeleton.pkl", "rb"))
    peps_params = pickle.load(open(f"../../data/{Lx}x{Ly}/t={t}_U={U}/N={N_f}/{symmetry}/D={D}/peps_su_params.pkl", "rb"))
    peps = qtn.unpack(peps_params, skeleton)
except:
    peps = generate_random_fpeps(Lx, Ly, D=D, seed=2, symmetry=symmetry, Nf=N_f, spinless=False)[0]
peps_np = peps.copy()
peps.apply_to_arrays(lambda x: torch.tensor(x, dtype=dtype))

# VMC sample size
N_samples = 2
N_samples = closest_divisible(N_samples, SIZE)

In [2]:
random_x = torch.tensor(H.hilbert.random_state())
random_x1 = random_x.clone()
print(random_x)
random_x1[1] = 2
random_x1[5] = 1

print((peps.product_bra_state(random_x).conj()|peps).contract(), (peps|peps.product_bra_state(random_x, reverse=1)).contract())

permute_list = []
for tensor in peps.product_bra_state(random_x).conj().tensors:
    # print(tensor.data.charge, tensor.data.oddpos)
    if len(tensor.data.oddpos)>0:
        # print(tensor.data.oddpos[0].label, tensor.data.oddpos[0].dual)
        permute_list.append(tensor.data.oddpos[0].label)

# sort the list and note down the parity of the permutation
print(len(permute_list))
# sum from len(permute_list) to 1
N = 0
for i in range(len(permute_list)):
    N += i
print('phase correction from reversing the ordering:', (-1)**(N%2))

tensor([3, 2, 0, 1, 3, 0, 3, 0, 3, 1, 2, 0, 0, 0, 3, 0])
tensor(3809.5152, dtype=torch.float64) tensor(3809.5152, dtype=torch.float64)
4
phase correction from reversing the ordering: 1


In [9]:
amp0 = peps.get_amp(random_x)
amp0.contract_boundary_from_xmax(xrange=(0, Lx-1), max_bond=16, cutoff=0.0, mode='zipup').contract(), amp0.contract()

(tensor(3809.5152, dtype=torch.float64),
 tensor(3809.5152, dtype=torch.float64))

In [10]:
amp0.contract_boundary_from_xmax(xrange=(0, Lx-1), max_bond=16, cutoff=0.0, mode='fit', tn_fit='zipup')

ImportError: autoray couldn't find function 'pad' for backend 'symmray'.